This notebook was prepared by [Author](https://github.com/). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Solution Notebook

## Problem: Implement foo(val), which returns val

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)

## Constraints

* Can we assume we have a Node Class?
    * No
* Can we assume the inputs are valid?
    * Yes
* Can we assume this fits memory?
    * Yes

## Test Cases

<table>
<tr>
<td>board</td>
<td>start</td>
<td>end</td>
<td>cost</td>
<td>expected</td>
</tr>
<tr>
<td>[]</td>
<td>[0, 0]</td>
<td>[3, 5]</td>
<td>1</td>
<td>None</td>
</tr>
<tr>
<td><pre>[[0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1],
[0, 1, 0, 1, 0, 0],
[0, 1, 0, 0, 1, 0],
[0, 0, 0, 0, 1, 0]]</pre></td>
<td>[0, 0]</td>
<td>[3, 5]</td>
<td>1</td>
<td><pre>[[0, -1, -1, -1, -1, -1],
[1,  2,  3,  4,  5, -1],
[-1, -1, -1, -1,  6,  7],
[-1, -1, -1, -1, -1,  8],
[-1, -1, -1, -1, -1, -1]]</pre></td>
</tr>
 <tr>
<td><pre>[[0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1],
[0, 1, 0, 1, 0, 0],
[0, 1, 0, 0, 1, 0],
[0, 0, 0, 0, 1, 0]]</pre></td>
<td>[0, 5]</td>
<td>[4, 0]</td>
<td>1</td>
<td><pre>[[-1, -1, -1, -1,  1,  0],
[-1, -1,  4,  3,  2, -1],
[-1, -1,  5, -1, -1, -1],
[-1, -1,  6, -1, -1, -1],
[ 9,  8,  7, -1, -1, -1]]</pre></td>
</tr>
<tr>
<td><pre>[[0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1],
[0, 1, 0, 1, 0, 0],
[0, 1, 0, 0, 1, 0],
[0, 0, 0, 0, 1, 0]]</pre></td>
<td>[0, 1]</td>
<td>[3, 5]</td>
<td>1</td>
<td>None</td>
</tr>
<tr>
<td><pre>[[0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1],
[0, 1, 0, 1, 0, 0],
[0, 1, 0, 0, 1, 0],
[0, 0, 0, 0, 1, 0]]</pre></td>
<td>[0, 0]</td>
<td>[2, 3]</td>
<td>1</td>
<td>None</td>
</tr>
</table>

## Algorithm
A* node have some characteristics :
    - parent: the previous node (called parent) of the current node
    - position: the current position of the node in the board
    - g : cost from start to current node
    - h: heuristic (estimated cost) from current node to the goal
    - f : total cost of current node equal to g+h
To find the path from start to an end:
* We add the start node to not_visited_nodes list
* while the not_visited_nodes is not empty:
    * if this node have a lowest f then it become the current_node so we add it to visited_nodes list and remove it from not_visited_nodes list so as we can go further with it children
    * We assume that the max_iteration isn't hit and the node is not the end node
    * if it is the end node, we return the path from start to this node, otherwise
    * we get all children and the current node become the parent of this nodes
    * we will use euclidean distance for the heuristic
    * if the child is already in not_visited_nodes list or have a highest g than others we ignore it and try the next child else we add it to not_visited_nodes list
    * We also ignore the child if its in danger zone

    
Complexity:
* Time: 2*O(log n)
* Space: O(b**d) b: average of successors per state, d the depth of the solution

## Code

In [1]:
from typing import List

import numpy as np


class Node:
    """
    A Node class for A* algorithm
    the previous node (called parent) of the current node  
    position: the current position of the node in the board
    g : cost from start to current node
    h: heuristic (estimated cost) from current node to the goal
    f : total cost of current node equal to g+h
    """

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position


class Astar(object):

    @staticmethod
    def get_path(current_node: Node, board):
        path = []
        board_rows, board_columns = np.shape(board)
        output = [[-1 for _ in range(board_columns)] for _ in range(board_rows)]
        while current_node is not None:
            path.append(current_node.position)
            current_node = current_node.parent  # node3 - node2 - node1 with node1 the parent of node2 and so on

        path = path[::-1]  # we reverse to get the logic path: start to end
        for i in range(len(path)):
            output[path[i][0]][path[i][1]] = i
        return output

    def search(self, board, cost: int, start: List[int], end: List[int]) -> List[List[int]]:
        """
        Return given the start node and the end one, the path for the given board
        :param board:
        :param cost:
        :param start:
        :param end:
        :return:
        """
        if len(board) == 0:
            return None

        if board[start[0]][start[1]] != 0:
            return None

        # INITIALISATION
        start_node, end_node = Node(None, tuple(start)), Node(None, tuple(end))
        start_node.g = start_node.h = start_node.f = 0
        end_node.g = end_node.h = end_node.f = 0

        visited_nodes, not_visited_nodes = [], []
        not_visited_nodes.append(start_node)  # Here we start

        # stop criteria to avoid an infinite search/loop
        _iterations = 0
        max_iteration = (len(board) // 2) ** 12

        # take the current node and compare all the f cost and selecting the lowest one. check
        # also if max_iteration is hit to stop searching

        while any(not_visited_nodes):
            _iterations += 1

            current_node = not_visited_nodes[0]
            current_index = 0

            for index, node in enumerate(not_visited_nodes):
                if node.f < current_node.f:
                    current_node = node
                    current_index = index

            if _iterations > max_iteration:
                print("Stopping searching...Too much iterations")
                return self.get_path(current_node, board)

            # Then we remove this node from not_visited_nodes, and add it to visited_nodes
            not_visited_nodes.pop(current_index)
            visited_nodes.append(current_node)

            if current_node == end_node:
                print(f"Goal reaches in {_iterations} iterations...")
                return self.get_path(current_node, board)

            # Until here the node is not the goal, so we go further with its children
            children = self.get_children(node=current_node, board=board)

            # We have children; let take one by one
            # if it's in visited_nodes we ignore it and try the next one
            # if it's in not_visited_nodes we ignore it, else we move it to that nodes list

            for child in children:
                if any([visited_child for visited_child in visited_nodes if visited_child == child]):
                    continue

                # f,g and h of the child
                child.g = current_node.g + cost
                # we use euclidean distance for the heuristic
                child.h = (((child.position[0] - end_node.position[0]) ** 2) + (
                        (child.position[1] - end_node.position[1]) ** 2))
                child.f = child.g + child.h

                if any([item for item in not_visited_nodes if item == child and child.g > item.g]):
                    continue

                not_visited_nodes.append(child)

    @staticmethod
    def get_children(node: Node, board) -> List[Node]:
        children = []
        moves = [
            [-1, 0],  # UP
            [1, 0],  # DOWN
            [0, 1],  # RIGHT
            [0, -1]  # LEFT
        ]
        board_rows, board_columns = np.shape(board)
        for move in moves:
            node_position = (node.position[0] + move[0],
                             node.position[1] + move[1])

            # check for board constraints
            if (node_position[0] > (board_rows - 1) or
                    node_position[0] < 0 or
                    node_position[1] > (board_columns - 1) or
                    node_position[1] < 0):
                continue

            # can't move to an danger zone
            # Here the value of danger zone in the given board is set to 1
            if board[node_position[0]][node_position[1]] != 0:
                continue
            child = Node(node, node_position)
            children.append(child)
        return children

## Unit Test

In [2]:
%%writefile test_astar.py

from unittest import TestCase


class TestAstar(TestCase):
    def test_astar_with_empty_board(self):
        board = []
        start = [0, 1]
        end = [3, 5]
        cost = 1  # cost is 1 per movement

        a_start = Astar()
        self.assertEqual(a_start.search(board=board, cost=cost, start=start, end=end), None)
        print('Success: test_astar_with_empty_board')

    def test_astar_with_valid_board(self):
        board = [
            [0, 1, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 1],
            [0, 1, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 0],
            [0, 0, 0, 0, 1, 0],
        ]
        start = [0, 0]
        end = [3, 5]
        cost = 1  # cost is 1 per movement
        expected = [
            [0, -1, -1, -1, -1, -1],
            [1, 2, 3, 4, 5, -1],
            [-1, -1, -1, -1, 6, 7],
            [-1, -1, -1, -1, -1, 8],
            [-1, -1, -1, -1, -1, -1]
        ]

        a_start = Astar()
        self.assertEqual(a_start.search(board=board, cost=cost, start=start, end=end), expected)
        print('Success: test_astar_with_valid_board')

    def test_astar_with_another_valid_board(self):
        board = [
            [0, 1, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 1],
            [0, 1, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 0],
            [0, 0, 0, 0, 1, 0],
        ]
        start = [0, 5]
        end = [4, 0]
        cost = 1  # cost is 1 per movement
        expected = [
            [-1, -1, -1, -1, 1, 0],
            [-1, -1, 4, 3, 2, -1],
            [-1, -1, 5, -1, -1, -1],
            [-1, -1, 6, -1, -1, -1],
            [9, 8, 7, -1, -1, -1]
        ]

        a_start = Astar()
        self.assertEqual(a_start.search(board=board, cost=cost, start=start, end=end), expected)
        print('Success: test_astar_with_another_valid_board')


    def test_astar_start_from_danger_zone(self):
        board = [
            [0, 1, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 1],
            [0, 1, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 0],
            [0, 0, 0, 0, 1, 0],
        ]
        start = [0, 1]
        end = [3, 5]
        cost = 1  # cost is 1 per movement

        a_start = Astar()
        self.assertEqual(a_start.search(board=board, cost=cost, start=start, end=end), None)
        print('Success: test_astar_start_from_danger_zone')


    def test_astar_with_danger_zone_as_end(self):
        board = [
            [0, 1, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 1],
            [0, 1, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 0],
            [0, 0, 0, 0, 1, 0],
        ]
        start = [0, 0]
        end = [2, 3]
        cost = 1
        expected = None

        a_start = Astar()
        self.assertEqual(a_start.search(board=board, cost=cost, start=start, end=end), expected)
        print('Success: test_astar_with_danger_zone_as_end')


def main():
    test = TestAstar()
    test.test_astar_with_empty_board()
    test.test_astar_with_valid_board()
    test.test_astar_with_another_valid_board()
    test.test_astar_start_from_danger_zone()
    test.test_astar_with_danger_zone_as_end()


if __name__ == '__main__':
    main()


Overwriting test_astar.py


In [3]:
%run -i test_astar.py

Success: test_astar_with_empty_board
Goal reaches in 9 iterations...
Success: test_astar_with_valid_board
Goal reaches in 10 iterations...
Success: test_astar_with_another_valid_board
Success: test_astar_start_from_danger_zone
Success: test_astar_with_danger_zone_as_end
